In [ ]:
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from bs4 import BeautifulSoup
import os
import json
import re
from bs4 import BeautifulSoup, Comment
from lxml import html
# %pip install --upgrade --quiet langchain langchain-community langchainhub langchain-chroma bs4
import boto3
from langchain_core.prompts import ChatPromptTemplate
from bs4 import BeautifulSoup
import os
import json
import re
from bs4 import BeautifulSoup, Comment
from langchain.chains import LLMChain
from langchain_aws import ChatBedrock
from langchain.globals import set_debug
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import minify_html
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from collections import OrderedDict
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
# !pip3 install -r requirements.txt

In [ ]:
def claude3_5_sonnet_config_new():

    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name="us-east-1",
        aws_access_key_id=os.getenv("KEY_ID"),
        aws_secret_access_key=os.getenv("ACCESS_KEY"),
        aws_session_token=os.getenv("SESSION_TOKEN"))
    
    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

    model_kwargs =  {
        "max_tokens": 4096,  # set maximum token generation
        "temperature": 0.7,  # increased for more creative outputs
        "top_k": 100,        # increased to allow more diverse token selection
        "top_p": 0.85,       # increased to encourage more creative sampling
        "stop_sequences": ["\n\nhuman"],
    }

    llm = ChatBedrock(
        client=bedrock_runtime,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )

    return llm

In [39]:
def extract_result(content, tag_name="csv_data"):
    pattern = r'<'+tag_name+'>(.*?)</'+tag_name+'>'
    match = re.search(pattern, content, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return None

In [40]:
def get_calude_response(system_message, human_message):
    
    # create prompt template
    messages = [
        ("system", system_message),
        ("human", "{human_input}")
    ]
    prompt = ChatPromptTemplate.from_messages(messages)

    # set llm model parameters
    llm = claude3_5_sonnet_config_new() # # previously 0.2 and top k 80 : mostly worked

    # create chain
    chain = prompt | llm | StrOutputParser() #| RunnableLambda(extract_result)

    # invoke chain
    response = chain.invoke({"human_input" : human_message})

    return response

In [ ]:
def get_prompt_msg(month, persona):

    system_message = "You are a synthetic data generator, and you can generate data according to the instructions provided to you which resembles to real life data."
        
    # read human_prompt from file
    with open("human_prompt.txt", "r") as file:
        human_message = """
            Generate user transaction data for US based citizen for """+month+""" month, consider on average 35 transactions. There should be some reoccurring transactions. Generate data in csv format. Include all the debit and credit transaction for an user, include salary credit as well. Make data realistic. Date must be in YYYY/MM/DD format. 

            Here is an example schema  
            User Id Txn ID Txn Amount Txn Date Txn Category Txn Mode Merchant Code Merchant Name Payment Network Issuer Name  
            US001 B002 788 21/06/25 shopping Credit Card CK32 Amazon VISA BOA 

            Generated data must be between <csv_data> tag.
            
            Following is user characterises, but you can add more characterises: 
            """+persona 

        human_message += file.read()

    return system_message, human_message

In [ ]:
personas_rahul = [
    "young age | Male| Average spending| | Party| middleclass| card holding 1| No EMI ",
    "Middle age | Male| Average spending| | Party| entertainment | food | middleclass| card holding 1| No EMI ",
    "young age | Male| Average spending| | fashion | nerd| fitness enthusiast |middleclass| card holding 1| No EMI",
    "old age | Female | high spending| | travel | nerd| rich| card holding 2| With emi or loan "
]

personas_krishna = [
    "Middle Age|  Male| luxury| Family person| upper middleclass| card holding 3| with EMI ",
    "old age | Male| Average spending| Party| middleclass| card holding 1| No EMI ",
    "young age | Female| Stingy spender| savings mindset| Middle class| Ed tech spender| card holding |NO EMI ",
    "young age | Male | Gambler | party | saving mindset | upper middle class | with emi  "
]

for i,persona in enumerate(personas_krishna):
    for month in ["January", "February", "March", "April", "May", "June"]:
        for duration in ["first", "last"]:
            system_message, human_message = get_prompt_msg(month+" "+duration+" 15 days", persona)

            response = get_calude_response(system_message, human_message)
            # print(response)

            # extract csv data from response
            csv_data = extract_result(response, tag_name="csv_data")

            # save response to file
            with open(f"data_generation/transaction_data/krishna/user_{i}_month_{month}"+"_"+duration+"_15_days"+".csv", "w") as f:
                f.write(csv_data)
            print("---------------------------\n Saved response for "+month+" month and persona "+str(i))

---------------------------
 Saved response for January month and persona 0
---------------------------
 Saved response for January month and persona 0
---------------------------
 Saved response for February month and persona 0
---------------------------
 Saved response for February month and persona 0
---------------------------
 Saved response for March month and persona 0
---------------------------
 Saved response for March month and persona 0
---------------------------
 Saved response for April month and persona 0
---------------------------
 Saved response for April month and persona 0
---------------------------
 Saved response for May month and persona 0
---------------------------
 Saved response for May month and persona 0
---------------------------
 Saved response for June month and persona 0
---------------------------
 Saved response for June month and persona 0
---------------------------
 Saved response for January month and persona 1
---------------------------
 Sav

In [ ]:
# print(response)

Here's a generated CSV dataset for a US-based citizen's transactions in January, based on the characteristics and categories you provided:

<csv_data>
User Id,Txn ID,Txn Amount,Txn Date,Txn Category,Txn Mode,Merchant Code,Merchant Name,Payment Network,Issuer Name
US001,A001,3500.00,2024/01/15,INCOME_WAGES,Direct Deposit,EMP01,TechCorp Inc.,ACH,Chase Bank
US001,B001,-45.67,2024/01/01,FOOD_AND_DRINK_GROCERIES,Credit Card,GR01,Whole Foods,VISA,Chase Bank
US001,B002,-12.99,2024/01/01,ENTERTAINMENT_MUSIC_AND_AUDIO,Credit Card,SP01,Spotify,VISA,Chase Bank
US001,B003,-14.99,2024/01/01,ENTERTAINMENT_TV_AND_MOVIES,Credit Card,NF01,Netflix,VISA,Chase Bank
US001,B004,-89.99,2024/01/02,GENERAL_MERCHANDISE_CLOTHING_AND_ACCESSORIES,Credit Card,AM01,Amazon,VISA,Chase Bank
US001,B005,-35.50,2024/01/03,FOOD_AND_DRINK_RESTAURANT,Credit Card,RS01,Cheesecake Factory,VISA,Chase Bank
US001,B006,-50.00,2024/01/04,TRANSPORTATION_GAS,Debit Card,SH01,Shell,VISA,Chase Bank
US001,B007,-15.99,2024/01/05,FOOD_AND_D

In [ ]:
import pandas as pd
import glob
import os

def combine_csv(file_start_with):
    # Pattern to match all CSVs starting with 'user_0'
    csv_files = glob.glob("data_generation/transaction_data/krishna/"+file_start_with+"*.csv")

    # List to hold DataFrames
    df_list = []

    # Read each CSV file (skip header after first file)
    for i, file in enumerate(csv_files):
        df = pd.read_csv(file)
        df_list.append(df)

    # Combine all DataFrames
    combined_df = pd.concat(df_list, ignore_index=True)

    # Save to a new CSV
    combined_df.to_csv("data_generation/transaction_data/krishna/"+file_start_with+".csv", index=False)


for file_start_with in ["user_0","user_1","user_2","user_3"]:
    combine_csv(file_start_with)

In [ ]:
# import pandas as pd
# import glob
# import os

# # Process each combined CSV file
# for file_start_with in ["user_0"]:
#     file_path = f"transaction_data/rahul/{file_start_with}.csv"
#     if not os.path.exists(file_path):
#         continue

#     df = pd.read_csv(file_path)

#     # Try to find the date column (case-insensitive)
#     date_col = None
#     for col in df.columns:
#         if col.strip().lower() in ["txn date", "date"]:
#             date_col = col
#             break

#     if date_col:
#         def fix_date(val):
#             val = str(val).strip()
#             # Try YYYY-MM-DD
#             if re.match(r"\d{4}-\d{2}-\d{2}", val):
#                 return pd.to_datetime(val, format="%Y-%m-%d").strftime("%Y/%m/%d")
#             # Try DD/MM/YY or DD/MM/YYYY
#             if re.match(r"\d{2}/\d{2}/\d{2,4}", val):
#                 try:
#                     # Try DD/MM/YYYY
#                     return pd.to_datetime(val, format="%d/%m/%Y").strftime("%Y/%m/%d")
#                 except:
#                     # Try DD/MM/YY
#                     return pd.to_datetime(val, format="%d/%m/%y").strftime("%Y/%m/%d")
#             # Try MM/DD/YYYY
#             if re.match(r"\d{2}/\d{2}/\d{4}", val):
#                 try:
#                     return pd.to_datetime(val, format="%m/%d/%Y").strftime("%Y/%m/%d")
#                 except:
#                     pass
#             # Already in YYYY/MM/DD
#             if re.match(r"\d{4}/\d{2}/\d{2}", val):
#                 return val
#             # If not matched, return as is
#             return val

#         df[date_col] = df[date_col].apply(fix_date)
#         df.to_csv(file_path, index=False)